Librerias

In [1]:
import os
import numpy as np
import cv2
import torchvision.models.segmentation
import torch
import torchvision.transforms as tf


Parametros de Entrenamiento


In [2]:
Learning_Rate=1e-5
width=height=900 # image width and height
batchSize=2

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

Lista de Todas las imagenes en el dataset

In [4]:
TrainFolder="LabPicsV1/LabPicsV1/Simple/Train//"
ListImages=os.listdir(os.path.join(TrainFolder, "Image")) 

Definimos las Transformaciones

In [5]:
transformImg=tf.Compose([tf.ToPILImage(),tf.Resize((height,width)), 
                         tf.ToTensor(),tf.Normalize((0.485, 0.456, 0.406), 
                                                    (0.229, 0.224, 0.225))])
transformAnn=tf.Compose([tf.ToPILImage(),tf.Resize((height,width)), tf.ToTensor()])

Lectura para el mapeamiento de la imagen con el mapa de segmentacion

In [6]:
def ReadRandomImage(): 
  idx=np.random.randint(0,len(ListImages)) # Pick random image 
  Img=cv2.imread(os.path.join(TrainFolder, "Image",ListImages[idx]))
  Filled =  cv2.imread(os.path.join(TrainFolder,   "Semantic/16_Filled", ListImages[idx].replace("jpg","png")),0)       
 
  Vessel =  cv2.imread(os.path.join(TrainFolder, "Semantic/1_Vessel", ListImages[idx].replace("jpg","png")),0)
  AnnMap = np.zeros(Img.shape[0:2],np.float32) # Segmentation map  
  if Vessel is not None:  AnnMap[ Vessel == 1 ] = 1    
  if Filled is not None:  AnnMap[ Filled  == 1 ] = 2
  Img=transformImg(Img)
  AnnMap=transformAnn(AnnMap)
  return Img,AnnMap

Funcion para la creacion del batch

In [7]:
def LoadBatch(): # Load batch of images
    images = torch.zeros([batchSize,3,height,width])
    ann = torch.zeros([batchSize, height, width])
    
    for i in range(batchSize):
        images[i],ann[i]=ReadRandomImage()
    
    return images, ann

Cargamos la Red Neuronal

In [8]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
Net = torchvision.models.segmentation.fcn_resnet50(pretrained=True)
Net.classifier[4] = torch.nn.Conv2d(256, 3, kernel_size=(1, 1), stride=(1, 1)) # Change final layer to 3 classes
Net=Net.to(device)
optimizer=torch.optim.Adam(params=Net.parameters(),lr=Learning_Rate) # Create adam optimizer

C:\Users\lsesp\anaconda3\envs\proyectos\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
C:\Users\lsesp\anaconda3\envs\proyectos\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=FCN_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=FCN_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Training Loop

In [10]:
for itr in range(20000): # Training loop
   images,ann=LoadBatch() 
   
   images=torch.autograd.Variable(images,requires_grad=False).to(device)    
   
   ann = torch.autograd.Variable(ann,requires_grad=False).to(device)              
   
   Pred=Net(images)['out'] # make prediction

RuntimeError: Given groups=1, weight of size [3, 256, 1, 1], expected input[2, 512, 113, 113] to have 256 channels, but got 512 channels instead

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)

In [ ]:
criterion = torch.nn.CrossEntropyLoss() # Set loss function
Loss=criterion(Pred,ann.long()) # Calculate cross entropy loss
Loss.backward() # Backpropogate loss
Optimizer.step() # Apply gradient descent change to weight

In [ ]:
if itr % 1000 == 0: 
   print("Saving Model" +str(itr) + ".torch")
   torch.save(Net.state_dict(), str(itr) + ".torch")